In [1]:
# Suzan Iloglu, May 21,2020
# Import packages
import csv
import gurobipy as gp
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import time
import requests
import io
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200
from IPython.display import Image


# MAPPING THE NEW POLITICS OF CARE: COMMUNITY HEALTH WORKERS
The project presents multiple options for how individual workers in such a Community Health Corps might be distributed within each state. It shows that what you choose to prioritize greatly impacts where care would be sent. We can define communities in greatest need in many ways: we can think about our current crisis and send people to where the COVID19 pandemic rages most fiercely; we can think of long term measures of social and economic inequality embedded in metrics like the Centers for Disease Control and Prevention’s Social Vulnerability Index; we can focus on the places with too many people dying too young and use the County Health Rankings Years-of-Potential-Life-Lost measure; we can think of joblessness and how the pandemic has thrown many into unemployment and target our resources in this way. 

The followings are our options to choose to define vulnerability:


- SOCIAL VULNERABILITY INDEX
- MEDICAID 
- UNEMPLOYMENT
- YEARS OF POTENTIAL LIFE LOST
- TOTAL COVID CASES
- COVID CASES BY POPULATION
- COVID DEATHS BY POPULATION

We will start with Social Vulnerability Index (SVI) from CDC website.

### I. Importing SVI data which includes the variables for calculating county SVI for each state
The CDC uses both a USA-wide and a state by state SVI scores. For our project given that funding is likely going to be managed at a state level, using a state by state SVI scores makes the most sense and will be most sensitive to regional socioeconomic differences. Even though the CDC SVI scores are calculated using percentile rankings, the data sets include raw data estimates for each variables. The following table shows the variablaes used in the method of calculating SVI scores. 




      American Community Survey (ACS), 2014-2018 (5-year) data for the following estimates:
<img src="Data/img/SVI_comp.png" width="500">


Note: Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a> 
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Later in the notebook, we will provide the formula to create the SVI value we use in our project. First, we import the data for the US mainland and Puerto Rico.

In [2]:
## import svi data downloaded from CDC website as cited above

## 48 state SVI scores by county
svi_counties_mainland = gpd.read_file("Data/SVI2018_US_COUNTY/SVI2018_US_county.shp")

## Puerto Rico SVI scores by county
svi_counties_puerto_rico = gpd.read_file("Data/PuertoRico_COUNTY/SVI2018_PuertoRico_county.shp")

## Merge 48 states and Puerto Rico SVI 
svi_counties = pd.concat([svi_counties_mainland,svi_counties_puerto_rico ], sort = False)


In [3]:
## Replacing -999 values with 0 for calculations
svi_county = svi_counties.fillna(0)
svi_county  = svi_county.replace(-999, 0)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)

In [4]:
## Create the list for State
State = svi_county.STATE.unique().tolist()

In [5]:
# Create a seperate dictionary for the variables to calculate SVI

# Persons below poverty estimate, 2014-2018 ACS
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Civilian (age 16+) unemployed estimate, 2014-2018 ACS
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.E_UNEMP))

# Per capita income estimate, 2014-2018 ACS
E_PCI = dict(zip(svi_county.FIPS, svi_county.E_PCI))

# Persons (age 25+) with no high school diploma estimate, 2014-2018 ACS
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.E_NOHSDP))

# Persons aged 65 and older estimate
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.E_AGE65))

# Persons aged 17 and younger estimate
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.E_AGE17))

# Population with a disability estimate
E_DISABL = dict(zip(svi_county.FIPS, svi_county.E_DISABL))

# Single parent households with children under 18 estimate
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.E_SNGPNT))

# Minority (all persons except white, nonHispanic) estimate, 2014-2018 ACS
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.E_MINRTY))

# Persons (age 5+) who speak English "less than well" estimate, 2014-2018 ACS
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.E_LIMENG))

# Housing in structures with 10 or more units estimate, 2014-2018 ACS
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.E_MUNIT))

# Mobile homes estimate MOE, 2014-2018 ACS
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.E_MOBILE))

# At household level (occupied housing units), more people than rooms estimate, 2014-2018 ACS
E_CROWD = dict(zip(svi_county.FIPS, svi_county.E_CROWD))

# Households with no vehicle available estimate, 2014-2018 ACS
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.E_NOVEH))

# Persons in institutionalized group quarters estimate, 2014-2018 ACS
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.E_GROUPQ))

# Percentage of persons below poverty estimate
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Medicaid 
Medicaid is a means-tested health insurance program for low-income children, pregnant women, adults, seniors, and people with disabilities. Medicaid is jointly funded by federal and state governments and managed by states within federal standards and a wide range of state options. <a href="https://data.medicaid.gov/Enrollment/State-Medicaid-and-CHIP-Applications-Eligibility-D/n5ce-jxme"> Data Source for Medicaid Enrollment </a> 

In [6]:

import sodapy
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.medicaid.gov", None)


# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("83yt-67it", limit=4000)


# Read the medicaid demand data
#df_mm = pd.read_csv("Data/2020_06_Preliminary_applications__eligibility_determinations__and_enrollment_data.csv")

# Convert to pandas DataFrame
df_mm = pd.DataFrame.from_records(results)
df_mm.head(5)
df_mm.columns


Index(['applications_for_financial_assistance_submitted_to_the_state_based_marketplace',
       'applications_for_financial_assistance_submitted_to_the_state_based_marketplace_footnotes',
       'final_report', 'geocoded_column',
       'individuals_determined_eligible_for_chip_at_application',
       'individuals_determined_eligible_for_chip_at_application_footnotes',
       'individuals_determined_eligible_for_medicaid_at_application',
       'individuals_determined_eligible_for_medicaid_at_application_footnotes',
       'latitude', 'longitude', 'medicaid_and_chip_child_enrollment',
       'medicaid_and_chip_child_enrollment_footnotes',
       'new_applications_submitted_to_medicaid_and_chip_agencies',
       'new_applications_submitted_to_medicaid_and_chip_agencies_footnotes',
       'preliminary_updated', 'report_date', 'state_abbreviation',
       'state_expanded_medicaid', 'state_name',
       'total_applications_for_financial_assistance_submitted_at_state_level',
       'total_a

In [7]:
df_mm['State Name'] = df_mm['state_name'].str.upper() 

In [8]:
Medicaid_state = dict(zip(df_mm['State Name'], df_mm['total_medicaid_and_chip_enrollment']))
Medicaid_state['PUERTO RICO'] = 1622194
print (Medicaid_state)

{'ALABAMA': '957116', 'ALASKA': '231145', 'ARIZONA': '1839932', 'ARKANSAS': '830467', 'CALIFORNIA': '11847711', 'COLORADO': '1337805', 'CONNECTICUT': '874974', 'DELAWARE': '239009', 'DISTRICT OF COLUMBIA': '248591', 'FLORIDA': '3892552', 'GEORGIA': '1928703', 'HAWAII': '351337', 'IDAHO': '340742', 'ILLINOIS': '2987496', 'INDIANA': '1602976', 'IOWA': '699741', 'KANSAS': '401103', 'KENTUCKY': '1416013', 'LOUISIANA': '1585024', 'MAINE': '232455', 'MARYLAND': '1372695', 'MASSACHUSETTS': '1616404', 'MICHIGAN': '2439425', 'MINNESOTA': '1085778', 'MISSISSIPPI': '632427', 'MISSOURI': '923641', 'MONTANA': '257006', 'NEBRASKA': '254159', 'NEVADA': '685073', 'NEW HAMPSHIRE': '193436', 'NEW JERSEY': '1759653', 'NEW MEXICO': '772102', 'NEW YORK': '6263164', 'NORTH CAROLINA': '1851558', 'NORTH DAKOTA': '96757', 'OHIO': '2788134', 'OKLAHOMA': '797220', 'OREGON': '1053931', 'PENNSYLVANIA': '3069309', 'RHODE ISLAND': '305208', 'SOUTH CAROLINA': '1048276', 'SOUTH DAKOTA': '114059', 'TENNESSEE': '1489536

In [9]:
df_mmm = pd.read_csv("Data/ACSST5Y2019.S2704_data_with_overlays_2020-12-11T225641.csv", header=[1])
df_mmm.head(1)
#df_mmm.dtypes

,id,Geographic Area Name,Estimate!!Total!!Civilian noninstitutionalized population,Margin of Error!!Total!!Civilian noninstitutionalized population,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,"Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)","Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)",Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALON

In [10]:
df_mmm['FIPS'] = df_mmm.id.astype(str).str[9:]
df_mmm.head(5)
df_mmm.FIPS.astype(int)
df_mmm['FIPS'] = pd.to_numeric(df_mmm['FIPS'])

In [11]:
# 2018 ACI data
# ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!Medicaid/means-tested public coverage alone or in combination"]))

#2019 ACI data
ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination"]))




In [12]:

#df_m = pd.read_csv("Data/Medicaid_Demand.csv")

# Unemployment 
The unemployment rate is calculated by the U.S. Bureau of Labor Statistics as the percentage of the civilian labor force who are without jobs and have actively sought work within the past four weeks. <a href="https://www.bls.gov/lau/laufaq.htm#Q01"> Data Source for Unemployment  </a> 

In [13]:
from io import StringIO
import datetime 
from datetime import date
from dateutil.relativedelta import relativedelta

back = date.today() + relativedelta(months= -3)
three_months_ago = back.strftime('%b-%y') 


url = 'https://www.bls.gov/web/metro/laucntycur14.txt'
s = requests.get(url).text



df_unemp = pd.read_csv(StringIO(s), sep='|',  skiprows=7, skipfooter=6, engine='python', names = ['LAUS Area Code', 'FIPS State', 'FIPS County', 'Area Title', 'Period', 'Civilian Labor Force','Employed','Unemployed_Level','Unemployed_Rate'])

df_unemp['Period'] = df_unemp['Period'].astype(str)

df_unemp['FIPS'] = df_unemp['LAUS Area Code'].str[3:8]

df_unemp = df_unemp[df_unemp['Period'].str.contains(str(three_months_ago))]

df_unemp.head(5)
#df_unemp.dtypes

,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
41846,CN0100100000000,1,1,"Autauga County, AL",Oct-20(p),"25,226","24,095","1,131",4.5,01001
41847,CN0100300000000,1,3,"Baldwin County, AL",Oct-20(p),"95,660","90,920","4,740",5.0,01003
41848,CN0100500000000,1,5,"Barbour County, AL",Oct-20(p),"9,634","8,880",754,7.8,01005
41849,CN0100700000000,1,7,"Bibb County, AL",Oct-20(p),"8,624","8,127",497,5.8,01007
41850,CN0100900000000,1,9,"Blount County, AL",Oct-20(p),"24,687","23,860",827,3.3,01009


In [14]:
#df_unemp[[ 'FIPS', 'Area Title', 'Period','Unemployed_Level','Unemployed_Rate']].to_csv('Data/County_employment.csv', index=False)


In [15]:


df_unemp.replace({'-', 0})

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.replace(',', '')

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].astype(str)

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.strip()

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].str.strip()

df_unemp['FIPS'] = pd.to_numeric(df_unemp['FIPS'])

df_unemp['Unemployed_Level'] = pd.to_numeric(df_unemp['Unemployed_Level'])

df_unemp['Unemployed_Rate'] = pd.to_numeric(df_unemp['Unemployed_Rate'])




# Fill NA with 0
#df_unemp = df_unemp.fillna(0)
df_unemp.tail(5)
#df_unemp.dtypes


,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
45060,CN7214500000000,72,145,"Vega Baja Municipio, PR",Oct-20(p),"12,543","11,146",1397,11.1,72145
45061,CN7214700000000,72,147,"Vieques Municipio, PR",Oct-20(p),"2,386","2,133",253,10.6,72147
45062,CN7214900000000,72,149,"Villalba Municipio, PR",Oct-20(p),"6,603","5,969",634,9.6,72149
45063,CN7215100000000,72,151,"Yabucoa Municipio, PR",Oct-20(p),"7,961","7,168",793,10.0,72151
45064,CN7215300000000,72,153,"Yauco Municipio, PR",Oct-20(p),"9,250","8,321",929,10.0,72153


In [16]:
 
## Read the Unemployment data using cvs data 
#df_unemp = pd.read_csv("Data/Unemployment.csv")

# Fill NA with 0
#df_unemp = df_unemp.fillna(0)

#df_unemp.head(5)

# Years of Potential Life Lost (YPLL)

Years of Potential Life Lost (YPLL) measures the rate of premature deaths by region. YPLL is calculated as the sum of the estimated number of years that individuals would have lived if they had not died before the age of 75 per 100,000 people. <a href="https://www.countyhealthrankings.org/sites/default/files/media/document/2020%20County%20Health%20Rankings%20Data%20-%20v2.xlsx"> Data Source for YPLL.  </a> More information about YPLL can be dounf in this <a href="https://www.countyhealthrankings.org/explore-health-rankings/measures-data-sources/county-health-rankings-model/health-outcomes/length-of-life/premature-death-ypll"> link. </a> 


In [17]:
# Read the YPLL data
df_y = pd.read_csv("Data/YPLL.csv")

# Fill NA with the mean of the data
df_y = df_y.fillna(df_y.mean())


# Population

In [18]:
# Read the Population data
df_pop = pd.read_csv("Data/County_pop_2019.csv")

# Fill NA with 0
df_pop = df_pop.fillna(0)


In [19]:
# Create a dictionary for the county and population
population_county = df_pop.set_index('FIPS')['pop'].to_dict()

# Create a dictionary for the county and YPLL
YPLL = dict(zip(df_y.FIPS, df_y.YPLL))

# Create a dictionary for the county and Unemployment
Unemployment = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Level))


# Create a dictionary for the county and Community Health Workers (CHW) demand
# Note that we assume a CHW can serve 55 Medicaid patient so the demand for CHW will be

#Medicaid_demand = dict(zip(df_m.FIPS, df_m.Med_Demand))


In [20]:
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

# COVID-19 Cases & COVID-19 Cases per Capita

What are COVID-19 Cases and COVID-19 Cases per Capita?

COVID-19 cases is an absolute metric of the total number of COVID-19 cases in a county over the last fourteen days.  COVID-19 cases per 100,000 is a relative metric calculated by dividing the number of COVID-19 cases by the estimated county population and multiplying by 100,000.  Cases include both confirmed cases, based on viral testing, and probable cases, based on specific criteria for symptoms and epidemiological exposure. We use NY Times Covid data. 


In [21]:
#### Data with the most recent date in NY Times dataset:

today = time.strftime('%Y-%m-%d')
covid_data_update_date = today#'2020-07-21'#today #or enter a specific date such as '2020-07-06'


In [22]:
## 14 day period defined
data_date_dt = pd.to_datetime(covid_data_update_date,infer_datetime_format = True)

N = 14

date_N_days_ago = data_date_dt - timedelta(days = N)

date_N1_days_ago = data_date_dt - timedelta(days = N+1)

In [23]:

# URL for mainland US data
url = "http://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
s = requests.get(url).content
covid = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [24]:
covid.tail(50)

,date,county,state,fips,cases,deaths
888012,2021-01-01,Ozaukee,Wisconsin,55089.0,7115,55.0
888013,2021-01-01,Pepin,Wisconsin,55091.0,699,5.0
888014,2021-01-01,Pierce,Wisconsin,55093.0,3612,35.0
888015,2021-01-01,Polk,Wisconsin,55095.0,3077,24.0
888016,2021-01-01,Portage,Wisconsin,55097.0,5952,54.0
888017,2021-01-01,Price,Wisconsin,55099.0,1015,5.0
888018,2021-01-01,Racine,Wisconsin,55101.0,19569,259.0
888019,2021-01-01,Richland,Wisconsin,55103.0,1134,13.0
888020,2021-01-01,Rock,Wisconsin,55105.0,13134,126.0
888021,2021-01-01,Rusk,Wisconsin,55107.0,1145,13.0


Note: Since NY data is seperately available, we first read the NY data for all 5 different borough then combine with the rest of US data.

In [25]:
# URL for NY
url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv"
#"https://raw.githubusercontent.com/nychealth/coronavirus-data/master/data-by-day.csv"

ny = requests.get(url).content
covid_ny = pd.read_csv(io.StringIO(ny.decode('utf-8')))


covid_ny.tail(5)

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,BX_PROBABLE_DEATH_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BX_HOSPITALIZED_COUNT_7DAY_AVG,BX_DEATH_COUNT_7DAY_AVG,BX_ALL_DEATH_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_PROBABLE_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BK_PROBABLE_DEATH_COUNT,BK_CASE_COUNT_7DAY_AVG,BK_ALL_CASE_COUNT_7DAY_AVG,BK_HOSPITALIZED_COUNT_7DAY_AVG,BK_DEATH_COUNT_7DAY_AVG,BK_ALL_DEATH_COUNT_7DAY_AVG,MN_CASE_COUNT,MN_PROBABLE_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,MN_PROBABLE_DEATH_COUNT,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,MN_HOSPITALIZED_COUNT_7DAY_AVG,MN_DEATH_COUNT_7DAY_AVG,MN_ALL_DEATH_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,QN_PROBABLE_DEATH_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,QN_HOSPITALIZED_COUNT_7DAY_AVG,QN_DEATH_COUNT_7DAY_AVG,QN_ALL_DEATH_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_PROBABLE_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,SI_ALL_DEATH_COUNT_7DAY_AVG,INCOMPLETE
301,12/26/2020,3287,923,257,32,8,3024,3834,251,37,42,523,151,48,8,0,492,624,48,6,6,980,251,83,6,1,924,1159,79,9,10,411,113,35,8,0,412,518,41,7,7,992,312,68,10,0,894,1152,64,10,10,381,96,23,0,0,303,382,19,5,5,6000
302,12/27/2020,2725,858,224,38,5,3089,3921,252,38,43,456,137,46,4,0,516,650,49,6,6,857,202,78,12,0,939,1174,81,10,11,294,128,33,9,0,414,524,41,7,7,818,321,55,5,1,915,1190,63,10,10,299,70,12,8,0,305,383,17,5,5,6000
303,12/28/2020,4985,1029,275,34,11,3208,4043,248,38,43,921,160,58,6,1,549,686,48,6,7,1483,281,79,9,0,973,1204,81,11,11,644,156,51,3,0,422,535,40,6,6,1392,368,69,9,2,938,1220,62,9,10,544,63,18,7,0,325,398,17,6,6,6000
304,12/29/2020,4273,984,254,32,17,3255,4097,250,35,43,824,170,63,8,0,566,702,51,6,7,1176,267,60,7,1,969,1199,79,9,10,498,141,49,6,1,418,533,42,6,6,1347,317,69,4,1,966,1252,63,8,9,427,88,13,7,0,336,410,16,5,5,6000
305,12/30/2020,2617,1042,162,19,16,3106,3964,236,33,42,583,182,48,4,0,563,702,51,6,6,703,292,36,4,1,920,1149,70,8,9,426,149,18,4,0,407,527,39,6,6,689,328,45,6,1,905,1199,61,8,9,215,90,15,1,0,312,387,15,5,5,6000


In [26]:
Kings = covid_ny[['date_of_interest', 'BK_CASE_COUNT', 'BK_DEATH_COUNT']]
Kings.rename(columns = {'BK_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BK_DEATH_COUNT': 'deaths'} , inplace=True)
Kings['county'] = 'Kings'
Kings['state'] = 'New York'
Kings['fips'] = 36047.0
#Kings.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [27]:
Bronx = covid_ny[['date_of_interest', 'BX_CASE_COUNT', 'BX_DEATH_COUNT']]
Bronx.rename(columns = {'BX_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BX_DEATH_COUNT': 'deaths'} , inplace=True)
Bronx['state'] = 'New York'
Bronx['county'] = 'Bronx'
Bronx['fips'] = 36005.0
#Bronx.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [28]:
Manhattan = covid_ny[['date_of_interest', 'MN_CASE_COUNT', 'MN_DEATH_COUNT']]
Manhattan.rename(columns = {'MN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'MN_DEATH_COUNT': 'deaths'} , inplace=True)
Manhattan['state'] = 'New York'
Manhattan['county'] = 'Manhattan'
Manhattan['fips'] = 36061.0
#Manhattan.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [29]:
Queens = covid_ny[['date_of_interest', 'QN_CASE_COUNT', 'QN_DEATH_COUNT']]
Queens.rename(columns = {'QN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'QN_DEATH_COUNT': 'deaths'} , inplace=True)
Queens['state'] = 'New York'
Queens['county'] = 'Queens'
Queens['fips'] = 36081.0
#Queens.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [30]:
Richmond = covid_ny[['date_of_interest', 'SI_CASE_COUNT', 'SI_DEATH_COUNT']]
Richmond.rename(columns = {'SI_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'SI_DEATH_COUNT': 'deaths'} , inplace=True)
Richmond['state'] = 'New York'
Richmond['county'] = 'Richmond'
Richmond['fips'] = 36085.0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [31]:
covid.head(5)


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [32]:
Bronx.tail(5)

,date,cases,deaths,state,county,fips
301,12/26/2020,523,8,New York,Bronx,36005.0
302,12/27/2020,456,4,New York,Bronx,36005.0
303,12/28/2020,921,6,New York,Bronx,36005.0
304,12/29/2020,824,8,New York,Bronx,36005.0
305,12/30/2020,583,4,New York,Bronx,36005.0


In [33]:
#Kings['deaths'] = Kings.groupby(by=['fips'])['deaths_d'].sum()
#Bronx['deaths'] = Bronx.groupby(by=['fips'])['deaths_d'].sum()
#Manhattan['deaths'] = Manhattan.groupby(by=['fips'])['deaths_d'].sum()
#Queens['deaths'] = Queens.groupby(by=['fips'])['deaths_d'].sum()
#Richmond['deaths'] = Richmond.groupby(by=['fips'])['deaths_d'].sum()

In [34]:
Kings.head(5)

,date,cases,deaths,county,state,fips
0,02/29/2020,0,0,Kings,New York,36047.0
1,03/01/2020,0,0,Kings,New York,36047.0
2,03/02/2020,0,0,Kings,New York,36047.0
3,03/03/2020,0,0,Kings,New York,36047.0
4,03/04/2020,1,0,Kings,New York,36047.0


In [35]:
#Kings = Kings.drop(['deaths_d'], axis=1)
#Bronx = Bronx.drop(['deaths_d'], axis=1)
#Manhattan = Manhattan.drop(['deaths_d'], axis=1)
#Queens = Queens.drop(['deaths_d'], axis=1)
#Richmond = Richmond.drop(['deaths_d'], axis=1)

In [36]:
Kings.tail(5)

,date,cases,deaths,county,state,fips
301,12/26/2020,980,6,Kings,New York,36047.0
302,12/27/2020,857,12,Kings,New York,36047.0
303,12/28/2020,1483,9,Kings,New York,36047.0
304,12/29/2020,1176,7,Kings,New York,36047.0
305,12/30/2020,703,4,Kings,New York,36047.0


In [37]:
covid['dt'] = pd.to_datetime(covid['date'], infer_datetime_format=True)
Kings['dt'] = pd.to_datetime(Kings['date'], infer_datetime_format=True)
Bronx['dt'] = pd.to_datetime(Bronx['date'], infer_datetime_format=True)
Manhattan['dt'] = pd.to_datetime(Manhattan['date'], infer_datetime_format=True)
Queens['dt'] = pd.to_datetime(Queens['date'], infer_datetime_format=True)
Richmond['dt'] = pd.to_datetime(Richmond['date'], infer_datetime_format=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [38]:

Kings_c = Kings[(Kings['dt']>date_N1_days_ago) & (Kings['dt']<= data_date_dt)].copy()
Queens_c = Queens[(Queens['dt']>date_N1_days_ago) & (Queens['dt']<= data_date_dt)].copy()
Bronx_c = Bronx[(Bronx['dt']>date_N1_days_ago) & (Bronx['dt']<= data_date_dt)].copy()
Manhattan_c = Manhattan[(Manhattan['dt']>date_N1_days_ago) & (Manhattan['dt']<= data_date_dt)].copy()
Richmond_c = Richmond[(Richmond['dt']>date_N1_days_ago) & (Richmond['dt']<= data_date_dt)].copy()

In [39]:
#Kings['cases'] = Kings.loc[(Kings['dt']<= data_date_dt)].groupby(['dt'])['d_cases'].sum()
#Kings['cases'] = Kings.apply(lambda x: x[(Kings['dt']<= data_date_dt)]['d_cases'].sum())

In [40]:
Kings.head(50)

,date,cases,deaths,county,state,fips,dt
0,02/29/2020,0,0,Kings,New York,36047.0,2020-02-29
1,03/01/2020,0,0,Kings,New York,36047.0,2020-03-01
2,03/02/2020,0,0,Kings,New York,36047.0,2020-03-02
3,03/03/2020,0,0,Kings,New York,36047.0,2020-03-03
4,03/04/2020,1,0,Kings,New York,36047.0,2020-03-04
5,03/05/2020,3,0,Kings,New York,36047.0,2020-03-05
6,03/06/2020,1,0,Kings,New York,36047.0,2020-03-06
7,03/07/2020,2,0,Kings,New York,36047.0,2020-03-07
8,03/08/2020,5,0,Kings,New York,36047.0,2020-03-08
9,03/09/2020,16,0,Kings,New York,36047.0,2020-03-09


In [41]:
K_c = Kings_c.groupby(['fips']).sum().reset_index()
Q_c = Queens_c.groupby(['fips']).sum().reset_index()
B_c = Bronx_c.groupby(['fips']).sum().reset_index()
M_c = Manhattan_c.groupby(['fips']).sum().reset_index()
R_c = Richmond_c.groupby(['fips']).sum().reset_index()
K_c.head(19)

,fips,cases,deaths
0,36047.0,11437,107


In [42]:
covid_death = covid[(covid['dt'] >= data_date_dt- timedelta(days = 1))]
Kings_death = Kings.groupby(by=['fips'])['deaths'].sum().reset_index()
Bronx_death = Bronx.groupby(by=['fips'])['deaths'].sum().reset_index()
Manhattan_death = Manhattan.groupby(by=['fips'])['deaths'].sum().reset_index()
Queens_death = Queens.groupby(by=['fips'])['deaths'].sum().reset_index()
Richmond_death = Richmond.groupby(by=['fips'])['deaths'].sum().reset_index()

In [43]:
# We merge the NY data with the rest of the US data
covid = pd.concat([covid, Kings, Bronx, Manhattan, Queens, Richmond], sort = False)


In [44]:
Queens.tail(14)

,date,cases,deaths,state,county,fips,dt
292,12/17/2020,479,10,New York,Queens,36081.0,2020-12-17
293,12/18/2020,898,12,New York,Queens,36081.0,2020-12-18
294,12/19/2020,774,8,New York,Queens,36081.0,2020-12-19
295,12/20/2020,673,6,New York,Queens,36081.0,2020-12-20
296,12/21/2020,1228,16,New York,Queens,36081.0,2020-12-21
297,12/22/2020,1149,10,New York,Queens,36081.0,2020-12-22
298,12/23/2020,1118,7,New York,Queens,36081.0,2020-12-23
299,12/24/2020,827,14,New York,Queens,36081.0,2020-12-24
300,12/25/2020,270,7,New York,Queens,36081.0,2020-12-25
301,12/26/2020,992,10,New York,Queens,36081.0,2020-12-26


In [45]:

covid.tail(50)

,date,county,state,fips,cases,deaths,dt
256,11/11/2020,Richmond,New York,36085.0,168,1.0,2020-11-11
257,11/12/2020,Richmond,New York,36085.0,161,2.0,2020-11-12
258,11/13/2020,Richmond,New York,36085.0,173,1.0,2020-11-13
259,11/14/2020,Richmond,New York,36085.0,112,1.0,2020-11-14
260,11/15/2020,Richmond,New York,36085.0,152,4.0,2020-11-15
261,11/16/2020,Richmond,New York,36085.0,183,2.0,2020-11-16
262,11/17/2020,Richmond,New York,36085.0,190,2.0,2020-11-17
263,11/18/2020,Richmond,New York,36085.0,197,2.0,2020-11-18
264,11/19/2020,Richmond,New York,36085.0,201,0.0,2020-11-19
265,11/20/2020,Richmond,New York,36085.0,221,1.0,2020-11-20


In [46]:
print (data_date_dt)

2021-01-02 00:00:00


In [47]:

covid_death = covid_death[['fips', 'deaths']]
covid_death.head(2)

,fips,deaths
884817,1001.0,50.0
884818,1003.0,169.0


In [48]:
Kings_death.head(2)

,fips,deaths
0,36047.0,6064


In [49]:
# We merge the NY data with the rest of the US data
covid_death = pd.concat([covid_death, Kings_death, Bronx_death, Manhattan_death, Queens_death, Richmond_death], sort = False)

covid_death = covid_death.rename(columns={"deaths": "total_deaths"})


# Create a dictionary for the cumulative COVID deaths in each county
County_covid_death = dict(zip(covid_death.fips, covid_death.total_deaths))

covid_death.tail(250)

,fips,total_deaths
887817,51690.0,27.0
887818,51115.0,4.0
887819,51117.0,39.0
887820,51119.0,12.0
887821,51121.0,42.0
887822,51125.0,4.0
887823,51127.0,5.0
887824,51700.0,70.0
887825,51710.0,108.0
887826,51131.0,31.0


In [50]:
covid_death.shape

(3250, 2)

In [51]:
print (County_covid_death)

{1001.0: 50.0, 1003.0: 169.0, 1005.0: 33.0, 1007.0: 46.0, 1009.0: 63.0, 1011.0: 22.0, 1013.0: 45.0, 1015.0: 157.0, 1017.0: 63.0, 1019.0: 22.0, 1021.0: 54.0, 1023.0: 22.0, 1025.0: 26.0, 1027.0: 34.0, 1029.0: 16.0, 1031.0: 38.0, 1033.0: 60.0, 1035.0: 17.0, 1037.0: 7.0, 1039.0: 40.0, 1041.0: 36.0, 1043.0: 78.0, 1045.0: 63.0, 1047.0: 43.0, 1049.0: 62.0, 1051.0: 83.0, 1053.0: 34.0, 1055.0: 96.0, 1057.0: 19.0, 1059.0: 37.0, 1061.0: 20.0, 1063.0: 20.0, 1065.0: 33.0, 1067.0: 8.0, 1069.0: 64.0, 1071.0: 34.0, 1073.0: 719.0, 1075.0: 15.0, 1077.0: 70.0, 1079.0: 43.0, 1081.0: 75.0, 1083.0: 58.0, 1085.0: 32.0, 1087.0: 30.0, 1089.0: 179.0, 1091.0: 26.0, 1093.0: 43.0, 1095.0: 86.0, 1097.0: 508.0, 1099.0: 11.0, 1101.0: 276.0, 1103.0: 89.0, 1105.0: 10.0, 1107.0: 26.0, 1109.0: 23.0, 1111.0: 30.0, 1113.0: 6.0, 1117.0: 103.0, 1115.0: 73.0, 1119.0: 24.0, 1121.0: 76.0, 1123.0: 98.0, 1125.0: 218.0, 1127.0: 138.0, 1129.0: 24.0, 1131.0: 19.0, 1133.0: 30.0, 2013.0: 0.0, 2016.0: 0.0, 2020.0: 118.0, 2050.0: 9.0, 2

In [52]:
## subset last last 15 days
covid_last15 = covid[(covid['dt']>date_N1_days_ago) & (covid['dt']<= data_date_dt)].copy()
covid_last15['dt_time_delta'] = covid_last15['dt']-data_date_dt

In [53]:
## calculate new daily cases

## sort values by county and date
covid_last15.sort_values(by=['fips','dt'],inplace=True)
## remove data with 'unknown' counties
covid_last15 = covid_last15[covid_last15['fips'].notnull()].copy()

## calculate daily difference in number of cases
covid_last15['new_cases'] = covid_last15.groupby('fips')['cases'].transform(lambda x: x.diff())
## set negative new cases to zero, this can occuer due to the disperacy in the data

covid_last15.loc[covid_last15.new_cases < 1e-6, 'new_cases'] = 0
covid_last15.sort_index(inplace = True)

In [54]:
covid_last15[covid_last15.county == 'New Haven'].head(50)

,date,county,state,fips,cases,deaths,dt,dt_time_delta,new_cases
842943,2020-12-19,New Haven,Connecticut,9009.0,41723,1425.0,2020-12-19,-14 days,NaN
846189,2020-12-20,New Haven,Connecticut,9009.0,41723,1425.0,2020-12-20,-13 days,0.0
849434,2020-12-21,New Haven,Connecticut,9009.0,42747,1449.0,2020-12-21,-12 days,1024.0
852679,2020-12-22,New Haven,Connecticut,9009.0,43180,1455.0,2020-12-22,-11 days,433.0
855924,2020-12-23,New Haven,Connecticut,9009.0,43568,1460.0,2020-12-23,-10 days,388.0
859169,2020-12-24,New Haven,Connecticut,9009.0,44134,1486.0,2020-12-24,-9 days,566.0
862414,2020-12-25,New Haven,Connecticut,9009.0,44134,1486.0,2020-12-25,-8 days,0.0
865659,2020-12-26,New Haven,Connecticut,9009.0,44134,1486.0,2020-12-26,-7 days,0.0
868904,2020-12-27,New Haven,Connecticut,9009.0,44134,1486.0,2020-12-27,-6 days,0.0
872149,2020-12-28,New Haven,Connecticut,9009.0,46237,1509.0,2020-12-28,-5 days,2103.0


In [55]:
## select just last 14 days now that we have daily new cases with 15th day as baseline
covid_last14 = covid_last15[(covid_last15['dt'] > date_N_days_ago) & (covid_last15['dt'] <= data_date_dt)].copy()


In [56]:
## group by FIPS to get case load and follow up demand values for each county
covid_last14_stats = covid_last14.groupby(['fips'])['new_cases'].sum().reset_index(name ='total_cases')

covid_last14_stats[covid_last14_stats.fips == 9009].head(5)

,fips,total_cases
310,9009.0,5652.0


In [57]:
covid.tail(50)

,date,county,state,fips,cases,deaths,dt
256,11/11/2020,Richmond,New York,36085.0,168,1.0,2020-11-11
257,11/12/2020,Richmond,New York,36085.0,161,2.0,2020-11-12
258,11/13/2020,Richmond,New York,36085.0,173,1.0,2020-11-13
259,11/14/2020,Richmond,New York,36085.0,112,1.0,2020-11-14
260,11/15/2020,Richmond,New York,36085.0,152,4.0,2020-11-15
261,11/16/2020,Richmond,New York,36085.0,183,2.0,2020-11-16
262,11/17/2020,Richmond,New York,36085.0,190,2.0,2020-11-17
263,11/18/2020,Richmond,New York,36085.0,197,2.0,2020-11-18
264,11/19/2020,Richmond,New York,36085.0,201,0.0,2020-11-19
265,11/20/2020,Richmond,New York,36085.0,221,1.0,2020-11-20


In [58]:
## group by FIPS to get case load and follow up demand values for each county
#covid_death = covid.groupby(['fips'])['deaths'].sum().reset_index(name ='total_deaths')
#covid_death.head(250)

In [59]:
## group by FIPS to get case load each month and follow up demand values for each county
covid['month'] = pd.to_datetime(covid['date'])
covid['year'] = pd.to_datetime(covid['date'])


In [60]:
#covid_last14_stats_montly = covid.groupby(['fips', covid.month.dt.month, covid.year.dt.year])['deaths'].sum().reset_index() 

#covid_last14_stats_montly['cumulative_death'] = covid_last14_stats_montly.groupby(['fips'])['deaths'].cumsum(axis = 0) 
#covid_last14_stats_montly = covid_last14_stats_montly.groupby(['fips', covid_last14_stats_montly.month,  covid_last14_stats_montly.year])['deaths'].cumsum()

#covid_last14_stats_montly.head(10)

In [61]:
# adding population information from CDC svi dataset
covid_last14_stats = covid_last14_stats.reset_index()
covid_last14_stats['fips'] = covid_last14_stats['fips'].astype(int)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
covid_last14_stats = pd.merge(left = covid_last14_stats, right = svi_county[['E_TOTPOP','FIPS', 'STATE']], how = 'right', right_on = 'FIPS', left_on = 'fips' )
covid_last14_stats.fillna(0 , inplace=True)

In [62]:

covid_last14_stats.tail(5)

,index,fips,total_cases,E_TOTPOP,FIPS,STATE
3215,3217.0,72149.0,37.0,22993,72149,PUERTO RICO
3216,3218.0,72151.0,83.0,34149,72151,PUERTO RICO
3217,3219.0,72153.0,42.0,36439,72153,PUERTO RICO
3218,0.0,0.0,0.0,2132,2105,ALASKA
3219,0.0,0.0,0.0,689,2282,ALASKA


In [63]:
now = pd.to_datetime("now")

print (now)
m_now = now.month
print (m_now)

y_now = now.year

2021-01-03 00:21:33.845990
1


In [64]:
#one_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-1) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#one_month_lag_death = dict(zip(one_month_lag_cumulative_death.fips, one_month_lag_cumulative_death.cumulative_death))

#one_month_lag_cumulative_death.head(5)


In [65]:
#two_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-2) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#two_month_lag_death = dict(zip(two_month_lag_cumulative_death.fips, two_month_lag_cumulative_death.cumulative_death))


#two_month_lag_cumulative_death.head(5)

In [66]:
#three_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now - 3) & (covid_last14_stats_montly['year'] == y_now)]


# Create a dictionary for the 3 last month COVID deaths in each county
#three_month_lag_death = dict(zip(three_month_lag_cumulative_death.fips, three_month_lag_cumulative_death.cumulative_death))

#three_month_lag_cumulative_death.head(5)

#print (three_month_lag_death)

In [67]:

# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_county.FIPS, svi_county.STATE))

# Create a dictionary for the name of the given the county FIPS
county_name = dict(zip(svi_county.FIPS, svi_county.COUNTY))

# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = svi_county.FIPS.tolist() #[k for k in SVI_county] #[9001, 9003, 9005, 9007, 9009, 9011, 9013, 9015]#[k for k in SVI_county]




In [68]:
for j in County_covid_death:
       
    print (j, County_covid_death[j])

1001.0 50.0
1003.0 169.0
1005.0 33.0
1007.0 46.0
1009.0 63.0
1011.0 22.0
1013.0 45.0
1015.0 157.0
1017.0 63.0
1019.0 22.0
1021.0 54.0
1023.0 22.0
1025.0 26.0
1027.0 34.0
1029.0 16.0
1031.0 38.0
1033.0 60.0
1035.0 17.0
1037.0 7.0
1039.0 40.0
1041.0 36.0
1043.0 78.0
1045.0 63.0
1047.0 43.0
1049.0 62.0
1051.0 83.0
1053.0 34.0
1055.0 96.0
1057.0 19.0
1059.0 37.0
1061.0 20.0
1063.0 20.0
1065.0 33.0
1067.0 8.0
1069.0 64.0
1071.0 34.0
1073.0 719.0
1075.0 15.0
1077.0 70.0
1079.0 43.0
1081.0 75.0
1083.0 58.0
1085.0 32.0
1087.0 30.0
1089.0 179.0
1091.0 26.0
1093.0 43.0
1095.0 86.0
1097.0 508.0
1099.0 11.0
1101.0 276.0
1103.0 89.0
1105.0 10.0
1107.0 26.0
1109.0 23.0
1111.0 30.0
1113.0 6.0
1117.0 103.0
1115.0 73.0
1119.0 24.0
1121.0 76.0
1123.0 98.0
1125.0 218.0
1127.0 138.0
1129.0 24.0
1131.0 19.0
1133.0 30.0
2013.0 0.0
2016.0 0.0
2020.0 118.0
2050.0 9.0
2060.0 0.0
2068.0 0.0
2070.0 1.0
2090.0 19.0
2100.0 0.0
2110.0 3.0
2122.0 15.0
2130.0 0.0
2150.0 2.0
2158.0 2.0
2164.0 0.0
2170.0 16.0
2180.0 0.

22071.0 679.0
22073.0 284.0
22075.0 21.0
22077.0 57.0
22079.0 212.0
22081.0 29.0
22083.0 40.0
22085.0 44.0
22087.0 39.0
22089.0 73.0
22091.0 5.0
22093.0 43.0
22095.0 119.0
22097.0 202.0
22099.0 82.0
22101.0 102.0
22103.0 345.0
22105.0 184.0
22107.0 3.0
22109.0 149.0
22111.0 66.0
nan 0.0
22113.0 85.0
22115.0 76.0
22117.0 87.0
22119.0 68.0
22121.0 48.0
22123.0 34.0
22125.0 25.0
22127.0 30.0
23001.0 44.0
23003.0 8.0
23005.0 97.0
23007.0 5.0
23009.0 12.0
23011.0 29.0
23013.0 4.0
23015.0 2.0
23017.0 21.0
23019.0 20.0
23021.0 1.0
23023.0 1.0
23025.0 21.0
nan 0.0
23027.0 17.0
23029.0 2.0
23031.0 63.0
24001.0 154.0
24003.0 367.0
24005.0 942.0
24510.0 711.0
24009.0 46.0
24011.0 10.0
24013.0 167.0
24015.0 75.0
24017.0 124.0
24019.0 22.0
24021.0 196.0
24023.0 53.0
24025.0 168.0
24027.0 181.0
24029.0 27.0
24031.0 1129.0
24033.0 1089.0
24035.0 28.0
24039.0 22.0
24037.0 89.0
24041.0 8.0
nan 55.0
24043.0 153.0
24045.0 78.0
24047.0 48.0
25001.0 238.0
25003.0 138.0
25005.0 1071.0
25007.0 0.0
25009.0 16

48447.0 3.0
48449.0 45.0
48451.0 180.0
48453.0 549.0
48455.0 9.0
48457.0 11.0
48459.0 31.0
48461.0 7.0
48463.0 42.0
48465.0 153.0
48467.0 63.0
48469.0 122.0
48471.0 86.0
48473.0 22.0
48475.0 9.0
48477.0 60.0
48479.0 509.0
48481.0 74.0
48483.0 7.0
48485.0 218.0
48487.0 31.0
48489.0 63.0
48491.0 213.0
48493.0 36.0
48495.0 15.0
48497.0 72.0
48499.0 68.0
48501.0 20.0
48503.0 30.0
48505.0 10.0
48507.0 25.0
49001.0 2.0
49003.0 26.0
49005.0 20.0
49007.0 7.0
49009.0 0.0
49011.0 86.0
49013.0 6.0
49015.0 6.0
49017.0 7.0
49019.0 1.0
49021.0 15.0
49023.0 9.0
49025.0 2.0
49027.0 2.0
49029.0 7.0
49031.0 2.0
49033.0 0.0
49035.0 554.0
49037.0 32.0
49039.0 12.0
49041.0 8.0
49043.0 5.0
49045.0 13.0
49047.0 9.0
nan 2.0
49049.0 215.0
49051.0 15.0
49053.0 111.0
49055.0 0.0
49057.0 99.0
50001.0 4.0
50003.0 3.0
50005.0 3.0
50007.0 69.0
50009.0 0.0
50011.0 25.0
50013.0 0.0
50015.0 1.0
50017.0 1.0
50019.0 5.0
50021.0 8.0
nan 0.0
50023.0 9.0
50025.0 5.0
50027.0 3.0
78010.0 7.0
78020.0 1.0
78030.0 15.0
51001.0 2

In [69]:
K_c.head()

,fips,cases,deaths
0,36047.0,11437,107


In [70]:
K = dict(zip(K_c.fips, K_c.cases))
Q = dict(zip(Q_c.fips, Q_c.cases))
B = dict(zip(B_c.fips, B_c.cases))
M = dict(zip(M_c.fips, M_c.cases))
R  = dict(zip(R_c.fips, R_c.cases))

In [71]:
print (K)

{36047.0: 11437}


In [72]:
# Create a dictionary for the county and covid cases
covid_cases_county_ny_times = dict(zip(covid_last14_stats.fips, covid_last14_stats.total_cases))
COVID_14days = {}

for j in location:
    if j in covid_cases_county_ny_times:
        COVID_14days[j] = covid_cases_county_ny_times[j] 
        print(j, covid_cases_county_ny_times[j])
        

        
    else:
        COVID_14days[j] = 0
        
        
    if j not in County_covid_death:
        print ('j',j)
        County_covid_death[j] = 0

#print (three_month_lag_death)
#print (two_month_lag_covid_death)
#print (one_month_lag_covid_death)
#for j in location:
#    if j not in three_month_lag_death:        
#        three_month_lag_death[j] = 0
        
#    if j not in two_month_lag_death:        
#        two_month_lag_death[j] = 0
        
#    if j not in one_month_lag_death:        
#        one_month_lag_death[j] = 0
        
    #if j not in County_covid_death:
    #    County_covid_death[j] = 0

35039 316.0
1001 541.0
1009 502.0
1013 131.0
1015 990.0
1017 250.0
1031 488.0
1033 611.0
1039 235.0
1043 953.0
1045 381.0
1051 1070.0
1055 1287.0
1067 187.0
1069 844.0
1071 542.0
1077 1044.0
1079 343.0
1083 956.0
1089 3528.0
1095 947.0
1097 2927.0
1103 1541.0
1111 96.0
1113 248.0
1115 855.0
1117 2114.0
1121 585.0
2261 34.0
4021 4654.0
5009 281.0
5011 158.0
5033 569.0
5037 217.0
5045 1300.0
5047 119.0
5051 902.0
5053 256.0
5063 325.0
5083 194.0
5085 651.0
5087 143.0
5115 823.0
5117 106.0
5121 183.0
5125 1066.0
5131 1230.0
5145 901.0
6007 1499.0
6017 1168.0
6023 377.0
6027 137.0
6061 2760.0
6079 2248.0
6089 1075.0
6093 157.0
8019 35.0
8039 111.0
8047 12.0
8051 115.0
8057 4.0
8065 46.0
8067 205.0
8071 105.0
8075 219.0
8077 1098.0
8083 124.0
8085 328.0
8093 45.0
8099 80.0
8101 1092.0
8103 20.0
8119 78.0
8121 19.0
9005 961.0
9007 1199.0
9011 1678.0
9015 987.0
10001 1500.0
12019 1451.0
12033 2322.0
12089 744.0
12101 2641.0
12109 1900.0
12113 1578.0
13015 1288.0
13039 317.0
13045 776.0
13047 

40149 122.0
40153 216.0
41007 94.0
41009 113.0
41017 576.0
41029 669.0
41035 320.0
41037 19.0
41043 383.0
41053 272.0
41061 39.0
41065 149.0
41071 318.0
42001 1080.0
42005 693.0
42007 1298.0
42009 332.0
42013 1056.0
42015 405.0
42017 4504.0
42019 1865.0
42025 663.0
42029 3067.0
42031 295.0
42033 906.0
42035 455.0
42037 631.0
42039 721.0
42041 1786.0
42047 399.0
42051 1751.0
42055 1295.0
42057 146.0
42063 371.0
42065 312.0
42067 198.0
42073 631.0
42075 1371.0
42077 3674.0
42081 1443.0
42083 437.0
42085 936.0
42089 1058.0
42095 3085.0
42099 414.0
42103 256.0
42105 116.0
42107 1635.0
42109 287.0
42111 1037.0
42115 193.0
42117 199.0
42121 473.0
42123 499.0
42125 1613.0
42127 593.0
42129 3248.0
42131 153.0
42133 4142.0
44001 587.0
44005 630.0
44009 769.0
45003 997.0
45007 1635.0
45015 830.0
45023 260.0
45035 1085.0
45045 5651.0
45055 412.0
45057 711.0
45063 2307.0
45073 722.0
45077 1353.0
45083 2885.0
45091 1985.0
46015 6.0
46019 50.0
46025 13.0
46029 190.0
46039 21.0
46043 21.0
46045 51.0


53031 16.0
53039 61.0
53043 31.0
53053 3316.0
53055 0.0
53061 2510.0
53063 2578.0
53071 534.0
53073 376.0
54001 187.0
54007 99.0
54011 983.0
54017 57.0
54019 307.0
54027 235.0
54039 1155.0
54061 831.0
54063 140.0
54069 358.0
54071 99.0
54077 432.0
54081 469.0
54093 75.0
55001 75.0
55007 67.0
55013 49.0
55019 219.0
55025 2057.0
55029 166.0
55033 281.0
55037 13.0
55051 21.0
55059 1030.0
55063 717.0
55085 140.0
55099 56.0
55113 128.0
55125 114.0
55129 95.0
55133 2440.0
55139 825.0
56001 98.0
56005 284.0
56017 24.0
56023 102.0
56027 3.0
56037 264.0
56041 152.0
1007 253.0
1037 65.0
1053 303.0
1101 1545.0
1107 202.0
1127 597.0
1129 113.0
2020 1391.0
2100 0.0
2130 10.0
4005 1487.0
4011 49.0
5001 145.0
5013 45.0
5021 135.0
5027 187.0
5119 3723.0
5129 71.0
5135 187.0
5141 117.0
5149 289.0
6005 766.0
6015 142.0
6051 139.0
6055 1113.0
6087 2071.0
6097 2634.0
6103 513.0
6111 10947.0
8033 4.0
8037 322.0
8045 494.0
8053 2.0
8055 24.0
8073 46.0
8087 147.0
8109 11.0
8111 3.0
8117 209.0
8125 60.0
9001 

37107 345.0
37163 471.0
37181 301.0
37187 61.0
39075 149.0
40091 199.0
41045 181.0
45025 240.0
45027 224.0
45049 105.0
45065 62.0
47029 455.0
47129 250.0
48055 206.0
48105 34.0
48115 57.0
48137 6.0
48165 62.0
48273 146.0
48291 448.0
48313 63.0
48329 1484.0
48345 14.0
48409 217.0
48413 9.0
48439 19867.0
48443 9.0
48449 226.0
48457 96.0
48471 591.0
48473 336.0
48495 61.0
49039 441.0
51029 206.0
51081 51.0
51153 3522.0
51590 312.0
51620 56.0
51660 366.0
51683 275.0
51685 87.0
51690 128.0
51700 977.0
51740 719.0
51760 1203.0
53025 634.0
54013 21.0
54109 202.0
1065 213.0
2013 0.0
2185 59.0
4009 519.0
4012 365.0
5035 410.0
5095 83.0
5099 52.0
6039 2000.0
6071 41460.0
6075 3425.0
6077 9280.0
6099 5480.0
12013 172.0
12077 78.0
12079 77.0
12086 26775.0
12093 190.0
12123 148.0
12125 80.0
13019 203.0
13063 1761.0
13095 495.0
13099 54.0
13109 61.0
13215 1346.0
13261 196.0
13267 197.0
13301 32.0
13303 150.0
16065 231.0
20057 193.0
21013 147.0
21045 51.0
21065 66.0
21075 23.0
21109 67.0
21121 222.0


In [73]:
for j in location: 
    if j in K:
        COVID_14days[j] = K[j]
        print (j, COVID_14days[j], K[j])

    if j in Q:
        COVID_14days[j] = Q[j]
        print (j, COVID_14days[j])
        
    if j in B:
        COVID_14days[j] = B[j]
        print (j, COVID_14days[j])
        
    if j in M:
        COVID_14days[j] = M[j]
        print (j, COVID_14days[j])

    if j in R:
        COVID_14days[j] = R[j]
        print (j, COVID_14days[j])

36085 3856
36061 5074
36047 11437 11437
36081 11277
36005 6610


In [74]:
print (covid_cases_county_ny_times)

{1001.0: 541.0, 1003.0: 2101.0, 1005.0: 187.0, 1007.0: 253.0, 1009.0: 502.0, 1011.0: 136.0, 1013.0: 131.0, 1015.0: 990.0, 1017.0: 250.0, 1019.0: 151.0, 1021.0: 328.0, 1023.0: 18.0, 1025.0: 244.0, 1027.0: 98.0, 1029.0: 93.0, 1031.0: 488.0, 1033.0: 611.0, 1035.0: 36.0, 1037.0: 65.0, 1039.0: 235.0, 1041.0: 131.0, 1043.0: 953.0, 1045.0: 381.0, 1047.0: 232.0, 1049.0: 618.0, 1051.0: 1070.0, 1053.0: 303.0, 1055.0: 1287.0, 1057.0: 147.0, 1059.0: 289.0, 1061.0: 208.0, 1063.0: 96.0, 1065.0: 213.0, 1067.0: 187.0, 1069.0: 844.0, 1071.0: 542.0, 1073.0: 7966.0, 1075.0: 124.0, 1077.0: 1044.0, 1079.0: 343.0, 1081.0: 1167.0, 1083.0: 956.0, 1085.0: 58.0, 1087.0: 112.0, 1089.0: 3528.0, 1091.0: 216.0, 1093.0: 271.0, 1095.0: 947.0, 1097.0: 2927.0, 1099.0: 202.0, 1101.0: 1545.0, 1103.0: 1541.0, 1105.0: 93.0, 1107.0: 202.0, 1109.0: 216.0, 1111.0: 96.0, 1113.0: 248.0, 1115.0: 855.0, 1117.0: 2114.0, 1119.0: 119.0, 1121.0: 585.0, 1123.0: 220.0, 1125.0: 2180.0, 1127.0: 597.0, 1129.0: 113.0, 1131.0: 80.0, 1133.0:

In [75]:
#Parameters
pro_c_s = [(i,county_of_states[i]) for i in location ]
cartesian_pro_county_state = gp.tuplelist(pro_c_s)


In [76]:
df = covid_last14_stats
df['fips'] = df['fips'].astype(int)

In [77]:
###############################################################################################
######################## END calculating different types of vulnerabilities ###################

Since we allocate CHW proportional to the county values of certain vulnaribilites within state, we need a few function to help us with the calculations. 



In [78]:
# This function return the value for the state for the given dictionary

# More specifically sum upt the values for the counties of each state

def total_state(dict_1):
    state_dict = {}
    for s in State:
        state_dict [s] = sum(float(dict_1[j]) for (j,s) in cartesian_pro_county_state.select('*', s) if j in dict_1)  
    return state_dict



In [79]:
# Calculte the population per state by summing up the population in each county in the state
State_pop = total_state(population_county)

In [80]:


# This function returns the ratio of the dict value for county and state of the county
def Proportional(county_level, state_level):
    
    prop = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            prop[j] = (float(county_level[j])/float(state_level[s]))    
        else:
            prop[j] = 0
                
    return prop

In [81]:
print (ACI_total)
print (ACI_total[35039])

{1001: 9804, 1003: 33019, 1005: 7332, 1007: 4860, 1009: 8954, 1011: 3100, 1013: 5004, 1015: 25387, 1017: 7391, 1019: 5162, 1021: 9558, 1023: 3433, 1025: 6557, 1027: 3375, 1029: 2964, 1031: 9652, 1033: 9471, 1035: 2710, 1037: 2234, 1039: 6987, 1041: 3339, 1043: 15529, 1045: 10717, 1047: 13248, 1049: 17983, 1051: 12740, 1053: 6301, 1055: 21800, 1057: 4277, 1059: 6775, 1061: 6484, 1063: 2858, 1065: 4977, 1067: 3485, 1069: 23013, 1071: 11372, 1073: 125514, 1075: 3015, 1077: 15659, 1079: 6971, 1081: 23354, 1083: 15392, 1085: 3457, 1087: 5200, 1089: 48472, 1091: 4803, 1093: 6723, 1095: 21817, 1097: 93203, 1099: 5664, 1101: 52425, 1103: 21849, 1105: 3094, 1107: 4752, 1109: 6901, 1111: 4507, 1113: 13627, 1115: 13737, 1117: 19953, 1119: 4144, 1121: 19136, 1123: 9809, 1125: 34475, 1127: 13645, 1129: 4400, 1131: 4082, 1133: 5781, 2013: 451, 2016: 439, 2020: 51594, 2050: 9063, 2060: 120, 2068: 145, 2070: 2082, 2090: 13016, 2100: 731, 2105: 512, 2110: 4856, 2122: 12129, 2130: 2919, 2150: 2451, 2158

In [82]:
#ACI
ACI_State = total_state(ACI_total)

In [83]:
# Create dicts for the variables of SVI
E_POV = dict(zip(svi_county.FIPS, svi_county.EP_POV))
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.EP_UNEMP))
E_PCI = dict(zip(svi_county.FIPS, svi_county.EP_PCI))
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.EP_NOHSDP))
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.EP_AGE65))
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.EP_AGE17))
E_DISABL = dict(zip(svi_county.FIPS, svi_county.EP_DISABL))
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.EP_SNGPNT))
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.EP_MINRTY))
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.EP_LIMENG))
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.EP_MUNIT))
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.EP_MOBILE))
E_CROWD = dict(zip(svi_county.FIPS, svi_county.EP_CROWD))
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.EP_NOVEH))
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.EP_GROUPQ))


# Calculate the state value for the SVI variables
E_POV_State = total_state(E_POV)
E_UNEMP_State = total_state(E_UNEMP) 
E_PCI_State = total_state(E_PCI)
E_NOHSDP_State = total_state(E_NOHSDP) 
E_AGE65_State = total_state(E_AGE65)
E_AGE17_State = total_state(E_AGE17)
E_DISABL_State = total_state(E_DISABL)
E_SNGPNT_State = total_state(E_SNGPNT)
E_MINRTY_State = total_state(E_MINRTY)
E_LIMENG_State = total_state(E_LIMENG)
E_MUNIT_State = total_state(E_MUNIT)
E_MOBILE_State = total_state(E_MOBILE)
E_CROWD_State = total_state(E_CROWD)
E_NOVEH_State = total_state(E_NOVEH)
E_GROUPQ_State = total_state(E_GROUPQ)



# Calculate the proportinal values for the SVI variables
E_POV_Prop = Proportional(E_POV, E_POV_State )
E_UNEMP_Prop = Proportional(E_UNEMP, E_UNEMP_State ) 
E_PCI_Prop = Proportional(E_PCI, E_PCI_State )
E_NOHSDP_Prop = Proportional(E_NOHSDP, E_NOHSDP_State ) 
E_AGE65_Prop = Proportional(E_AGE65, E_AGE65_State )
E_AGE17_Prop = Proportional(E_AGE17, E_AGE17_State )
E_DISABL_Prop = Proportional(E_DISABL, E_DISABL_State )
E_SNGPNT_Prop = Proportional(E_SNGPNT, E_SNGPNT_State )
E_MINRTY_Prop = Proportional(E_MINRTY, E_MINRTY_State )
E_LIMENG_Prop = Proportional(E_LIMENG, E_LIMENG_State )
E_MUNIT_Prop = Proportional(E_MUNIT, E_MUNIT_State )
E_MOBILE_Prop = Proportional(E_MOBILE, E_MOBILE_State )
E_CROWD_Prop = Proportional(E_CROWD, E_CROWD_State )
E_NOVEH_Prop = Proportional(E_NOVEH, E_NOVEH_State )
E_GROUPQ_Prop = Proportional(E_GROUPQ, E_GROUPQ_State )

ACI_Prop = Proportional(ACI_total, ACI_State)


# SVI calculation 

We calculate the ratio of county value to state value by population for each SVI variables (we use EP-estimate percentage- values in the CDC data set), then we take the average of all 15 SVI variables. 

Let SVI variable set be K, where  

K = { Below Poverty, Unemployed, Income, No High School Diploma, Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households, Minority, Speaks English “Less than Well”, Multi-Unit Structures, Mobile Homes, Crowding, No Vehicle, Group Quarters }

We will use these variables in a county base and state base. While County base values are exactly same as the estimated values for these variables in the CDC website, to calculate the state base, we simply sum the county values for all of the counties in each state for each variable. Then we use the following formula to calculate the SVI value for each county.

Let $S$ is the set of states and $j$ is a county in the state $s$, where $s \in S$, $c^k_j$ SVI variable $k \in K$ value for county j, and $c_s$ SVI variable value for state s.

$SVI_j = \frac{1}{15}\sum_{k \in K} \frac{c^k_j}{c^k_s}$


In [84]:

from collections import Counter
# Sum all SVI variable values for each county
SVI_county_sum = dict(Counter(E_POV_Prop) + Counter(E_UNEMP_Prop) + Counter(E_PCI_Prop) + Counter(E_NOHSDP_Prop) + Counter(E_AGE65_Prop) + Counter(E_AGE17_Prop) + Counter(E_DISABL_Prop) + Counter(E_SNGPNT_Prop) + Counter(E_MINRTY_Prop) + Counter(E_LIMENG_Prop) + Counter(E_MUNIT_Prop) + Counter(E_MOBILE_Prop) + Counter(E_CROWD_Prop) + Counter(E_NOVEH_Prop) + Counter(E_GROUPQ_Prop))

# Divide the sum of all SVI variable values
SVI_county = {j: SVI_county_sum[j]/15 for j in SVI_county_sum }


# Proportional Allocation

We consider allocating 1 million CHW over the states proportional to Medicaid enrollment in each state. Further, we allocate CHW to counties in each state proportional to different county vulnerability criterias as follow.

- MEDICAID
- SVI
- YPLL
- UNEMPLOYMENT
- LAST 14 DAYS COVID CASES
- LAST 14 DAYS COVID CASES / POP
- COVID DEATHS / POP

To calculate the total number of allocated CHW to per county according to these vulnerability criterias, we define the following function called "Proportional_allocation", in which we multiply the CHW allocated to each state with the ratio of the chosen vulnerability criteria of the county to the chosen vulnerability criteria of the state, the function return a dictionary with the counties as keys and the number of CHW allocated to each county for the chosen vulnerability criteria as values. 

In [85]:
def Proportional_allocation(county_level, state_level, state_budget):
    prop_allocate = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            #print (j,s, county_level[j],state_level[s], state_budget[s])
            prop_allocate[j,s] = (float(county_level[j])/float(state_level[s]))*float(state_budget[s])
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

In [86]:
Medicaid_dem = Proportional_allocation(ACI_total, ACI_State, Medicaid_state)
Medicaid_demand = {m[0]: Medicaid_dem[m] for m in Medicaid_dem}
for s in State:
#    print (s)
#    print ( ACI_State[s])
    print (s,Medicaid_state[s])
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

NEW MEXICO 772102
ALABAMA 957116
ALASKA 231145
ARIZONA 1839932
ARKANSAS 830467
CALIFORNIA 11847711
COLORADO 1337805
CONNECTICUT 874974
DELAWARE 239009
FLORIDA 3892552
GEORGIA 1928703
IDAHO 340742
ILLINOIS 2987496
INDIANA 1602976
IOWA 699741
KANSAS 401103
KENTUCKY 1416013
LOUISIANA 1585024
MAINE 232455
MARYLAND 1372695
MASSACHUSETTS 1616404
MICHIGAN 2439425
MINNESOTA 1085778
MISSISSIPPI 632427
MISSOURI 923641
MONTANA 257006
NEBRASKA 254159
NEVADA 685073
NEW HAMPSHIRE 193436
NEW JERSEY 1759653
NEW YORK 6263164
NORTH CAROLINA 1851558
NORTH DAKOTA 96757
OHIO 2788134
OKLAHOMA 797220
OREGON 1053931
PENNSYLVANIA 3069309
RHODE ISLAND 305208
SOUTH CAROLINA 1048276
SOUTH DAKOTA 114059
TENNESSEE 1489536
TEXAS 4457644
UTAH 338812
VERMONT 161049
VIRGINIA 1497770
WASHINGTON 1780968
WEST VIRGINIA 521290
WISCONSIN 1112844
WYOMING 59302
HAWAII 351337
DISTRICT OF COLUMBIA 248591
PUERTO RICO 1622194


In [87]:
#print ('Med', Medicaid_demand[35039])
#print ('Med', Medicaid_demand[4017], 'Pop', population_county[4017], 'Med_capita', 100000*(Medicaid_demand[4017]/population_county[4017]))

In [88]:
#Further we create additional vulnerability values by considering SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita and the number of Medicaid enrolles in each county together

Covid_capita = {j: 100000*(COVID_14days[j]/population_county[j]) for j in location}

Covid_death_capita = {j: 100000*(County_covid_death[j]/population_county[j]) for j in location}

Medicaid_capita = {j: 100000*(Medicaid_demand[j]/population_county[j]) for j in location}

Unemployment_capita = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Rate))

#Three_month_lag_covid_death_capita = {j: 100000*(three_month_lag_death[j]/population_county[j]) for j in location}

#Two_month_lag_covid_death_capita = {j: 100000*(two_month_lag_death[j]/population_county[j]) for j in location}

#One_month_lag_covid_death_capita = {j: 100000*(one_month_lag_death[j]/population_county[j]) for j in location}

In [89]:
for j in location:
    print (j, County_covid_death[j], population_county[j], Covid_death_capita[j])

35039 42.0 38921 107.91089643123249
1001 50.0 55869 89.49506882170793
1009 63.0 57826 108.94753225192821
1013 45.0 19448 231.3862607980255
1015 157.0 113605 138.1981426873817
1017 63.0 33254 189.45089312563903
1031 38.0 52342 72.59944213060257
1033 60.0 55241 108.61497800546695
1039 40.0 37049 107.96512726389375
1043 78.0 83768 93.11431572915671
1045 63.0 49172 128.12169527373302
1051 83.0 81209 102.20542058146265
1055 96.0 102268 93.87100559314742
1067 8.0 17205 46.498111014240045
1069 64.0 105882 60.44464592659753
1071 34.0 51626 65.85828845930345
1077 70.0 92729 75.48878991469768
1079 43.0 32924 130.60381484631273
1083 58.0 98915 58.63620280038416
1089 179.0 372909 48.00098683593048
1095 86.0 96774 88.86684440035546
1097 508.0 413210 122.93990948912175
1103 89.0 119679 74.36559463230809
1111 30.0 22722 132.03063110641668
1113 6.0 57961 10.351788271423889
1115 73.0 89512 81.55331128787202
1117 103.0 217702 47.31238114486776
1121 76.0 79978 95.02613218635125
2261 2.0 9202 21.734405564

36095 5.0 30999 16.129552566211814
36097 10.0 17807 56.15769079575448
36101 120.0 95379 125.81385839650237
36103 2311.0 1476601 156.50808850867634
36107 54.0 48203 112.02622243428831
36111 143.0 177573 80.53026079415227
36113 32.0 63944 50.04378831477543
36115 16.0 61204 26.142082216848575
36117 39.0 89918 43.372850819635666
36121 23.0 39859 57.703404500865545
37001 117.0 169509 69.02288374068634
37009 27.0 27203 99.25375877660552
37021 169.0 261191 64.70360770470651
37023 75.0 90485 82.88666629828148
37025 146.0 216453 67.45113257843504
37027 28.0 82178 34.07237946895763
37031 17.0 69473 24.46993796151023
37035 158.0 159551 99.02789703605742
37037 65.0 74470 87.28346985363234
37049 83.0 102139 81.26180988652719
37053 11.0 27763 39.6210784137161
37055 5.0 37009 13.51022724202221
37057 85.0 167609 50.71326718732288
37059 13.0 42846 30.341222051066612
37069 37.0 69685 53.09607519552271
37071 239.0 224529 106.4450471876684
37077 63.0 60443 104.23043197723474
37087 61.0 62317 97.8866119999

46025 2.0 3736 53.53319057815846
46029 70.0 28009 249.91966867792493
46039 7.0 4351 160.88255573431394
46043 7.0 2921 239.64395754878467
46045 4.0 3829 104.46591799425438
46051 35.0 7052 496.3131026659104
46053 26.0 4185 621.2664277180406
46055 7.0 1899 368.6150605581885
46059 2.0 3191 62.676277029144465
46063 1.0 1298 77.04160246533128
46067 16.0 7291 219.4486353037992
46077 13.0 4939 263.2111763514881
46079 16.0 12797 125.02930374306477
46081 28.0 25844 108.34236186348862
46093 24.0 28332 84.7098686997035
46097 6.0 2216 270.7581227436823
46101 14.0 6576 212.89537712895378
46105 11.0 2865 383.94415357766144
46111 3.0 2344 127.98634812286689
46115 24.0 6376 376.41154328732745
46117 2.0 3098 64.55777921239509
46119 3.0 1391 215.6721782890007
46123 12.0 5441 220.54769343870612
46125 49.0 8384 584.4465648854962
46129 14.0 5435 257.58969641214355
47001 72.0 76978 93.53321728286004
47003 61.0 49713 122.70432281294632
47009 98.0 133088 73.63548930031257
47011 72.0 108110 66.5988345203959
470

18121 8.0 16937 47.23386668241129
18123 24.0 19169 125.20214930356305
18133 39.0 37576 103.78965296998085
18153 26.0 20669 125.79224926218008
18163 204.0 181451 112.42704642024567
18167 163.0 107038 152.28236700984698
19009 7.0 5496 127.36535662299855
19013 218.0 131228 166.12308348827995
19019 16.0 21175 75.5608028335301
19025 9.0 9668 93.09060819197352
19053 4.0 7870 50.825921219822106
19057 34.0 38967 87.25331690917957
19059 20.0 17258 115.88828369451848
19063 24.0 9208 260.6429192006951
19103 47.0 151140 31.096996162498343
19113 245.0 226706 108.06948206046599
19115 24.0 11035 217.48980516538285
19119 25.0 11755 212.6754572522331
19127 57.0 39369 144.783967080698
19137 15.0 9917 151.2554199858828
19153 412.0 490161 84.05401490530662
19157 24.0 18504 129.70168612191958
19167 44.0 34855 126.23726868455029
19191 18.0 19991 90.04051823320493
19193 167.0 103107 161.96766465904352
20003 0.0 7858 0.0
20005 10.0 16073 62.21613886642195
20021 16.0 19939 80.2447464767541
20023 10.0 2657 376.

37085 94.0 135976 69.12984644349002
37089 92.0 117417 78.35321972116473
37099 18.0 43938 40.96681687832856
37105 41.0 61779 66.3655934864598
37111 38.0 45756 83.04921758895009
37113 9.0 35858 25.099001617491215
37117 25.0 22440 111.40819964349376
37129 89.0 234473 37.95746205320016
37135 65.0 148476 43.77811902260298
37137 5.0 12726 39.28964325003929
37143 5.0 13463 37.13882492757929
37149 13.0 20724 62.72920285659139
37173 8.0 14271 56.05773947165581
37175 10.0 34385 29.08244874218409
37179 98.0 239859 40.8573370188319
37183 344.0 1111761 30.941902081472545
37191 145.0 123131 117.76075886657301
37193 72.0 68412 105.24469391334854
37195 103.0 81801 125.91533110842165
37199 5.0 18069 27.671702916597486
38013 1.0 2115 47.28132387706856
38015 176.0 95626 184.05036287202225
38017 169.0 181923 92.89644519934258
38019 4.0 3762 106.32642211589581
38023 2.0 2264 88.33922261484099
38029 10.0 3241 308.54674483184203
38035 66.0 69451 95.03102907085572
38037 7.0 2274 307.82761653474057
38039 2.0 2

37133 66.0 197938 33.34377431316877
37147 69.0 180742 38.17596352812296
38105 32.0 37589 85.131288408843
39009 9.0 65327 13.776845714635604
39035 897.0 1235072 72.62734480257022
39061 417.0 817473 51.01085907424465
39105 14.0 22907 61.11668922163531
39121 32.0 14424 221.85246810870774
39145 45.0 75314 59.74984730594577
39163 11.0 13085 84.06572411157813
40005 3.0 13758 21.805494984736153
40023 6.0 14672 40.89422028353326
40059 3.0 3688 81.34490238611713
40067 3.0 6002 49.983338887037654
40089 52.0 32832 158.3820662768031
40095 6.0 16931 35.43795404878625
40141 9.0 7250 124.13793103448276
41011 10.0 64487 15.50700141113713
41021 1.0 1912 52.30125523012552
41023 1.0 7199 13.890818169190165
41033 22.0 87487 25.146593208133783
41047 213.0 347818 61.23892380497846
41055 0.0 1780 0.0
41059 57.0 77950 73.12379730596535
41069 1.0 1332 75.07507507507508
42003 1036.0 1216045 85.19421567458441
42045 993.0 566747 175.2104554589614
45001 25.0 24527 101.92848697353936
45013 105.0 192122 54.652772717

5113 42.0 19964 210.37868162692845
5133 20.0 17007 117.59863585582409
6001 657.0 1671329 39.31003411057907
6013 347.0 1153526 30.081679996809783
6037 10552.0 10039107 105.10895042756293
6053 196.0 434061 45.15494365999249
6059 1875.0 3175692 59.0422496892016
6067 857.0 1552058 55.21700864271825
6081 227.0 766573 29.612313504389014
6083 160.0 446499 35.83434677345302
6085 748.0 1927852 38.79965889497741
6101 52.0 96971 53.62427942374524
6115 19.0 78668 24.152133014694666
8001 533.0 517421 103.01089441673221
8021 3.0 8205 36.56307129798903
8025 12.0 6061 197.98713083649562
12021 332.0 384902 86.25572223578989
12029 11.0 16826 65.37501485795792
12107 72.0 74521 96.61706096268166
13005 25.0 11164 223.93407380867075
13017 44.0 16700 263.47305389221555
13021 261.0 153159 170.41114136289738
13069 93.0 43273 214.91461188269824
13081 39.0 22372 174.3250491686036
13107 53.0 22646 234.0369160116577
13135 604.0 936250 64.51268357810413
13165 34.0 8676 391.88566159520514
13167 34.0 9643 352.5873690

22029 38.0 19259 197.3103484085363
22041 81.0 20015 404.69647764176864
22097 202.0 82124 245.96950952218594
22107 3.0 4334 69.2201199815413
22117 87.0 46194 188.3361475516301
28069 19.0 9742 195.031820981318
28125 17.0 4321 393.42744735015043
28151 116.0 43909 264.1827415791751
29069 15.0 29131 51.491538223885215
29133 16.0 13180 121.39605462822458
30005 22.0 6681 329.2920221523724
31173 9.0 7224 124.58471760797342
32015 7.0 5532 126.53651482284889
34013 2399.0 798975 300.25970775055544
35025 91.0 71070 128.0427747291403
35047 4.0 27277 14.664369248817684
37061 67.0 58741 114.06002621678213
37091 48.0 23677 202.72838619757573
37165 54.0 34823 155.06992504953624
37177 3.0 4016 74.7011952191235
38079 22.0 14176 155.19187358916477
45011 30.0 20866 143.77456148758748
45061 42.0 16828 249.58402662229616
46095 2.0 2061 97.0402717127608
47097 28.0 25633 109.2341903015644
48107 20.0 5737 348.6142583231654
48135 224.0 166223 134.75872773322584
48145 17.0 17297 98.28293923801814
48189 126.0 3340

In [90]:
County_covid_death[9009], population_county[9009], Covid_death_capita[9009]

(1529.0, 854757, 178.88124929073408)

In [91]:
# Dictionary for the total Covid per capita for each state
Total_covid_cap = total_state(Covid_capita) 


# Dictionary for the total Covid death per capita for each state
Total_covid_death_cap = total_state(Covid_death_capita) 


# Dictionary for the total Medicaid per capita for each state
Total_medicaid_cap = total_state(Medicaid_capita)


# Dictionary for the total Unemployment per capita for each state
Total_unemployment_cap = total_state(Unemployment_capita)



In [92]:
# Dictionaries for the different vulnerability criteria values for states

# Dictionary for total Medicaid patient numbers for each state
Medicaid_demand_state = total_state(Medicaid_demand) 

# Dictionary for total positive COVID cases for last 14 days in each state
Covid_state = total_state(COVID_14days) 

# Dictionary for total SVI values for each state
SVI_state = total_state(SVI_county) 

# Dictionary for total YPLL values for each state
YPLL_state = total_state(YPLL) 

# Dictionary for total Unemployment numbers for each state
Unemployment_state = total_state(Unemployment)  

In [93]:
print(Medicaid_demand_state)

{'NEW MEXICO': 772102.0, 'ALABAMA': 957116.0000000001, 'ALASKA': 231145.0, 'ARIZONA': 1839932.0, 'ARKANSAS': 830467.0000000003, 'CALIFORNIA': 11847711.0, 'COLORADO': 1337805.0, 'CONNECTICUT': 874974.0, 'DELAWARE': 239009.0, 'FLORIDA': 3892551.999999999, 'GEORGIA': 1928702.999999999, 'IDAHO': 340742.0000000001, 'ILLINOIS': 2987496.0000000005, 'INDIANA': 1602975.9999999995, 'IOWA': 699741.0, 'KANSAS': 401103.0000000001, 'KENTUCKY': 1416013.0000000002, 'LOUISIANA': 1585023.9999999995, 'MAINE': 232454.99999999994, 'MARYLAND': 1372695.0, 'MASSACHUSETTS': 1616403.9999999998, 'MICHIGAN': 2439425.0000000005, 'MINNESOTA': 1085778.0000000002, 'MISSISSIPPI': 632427.0000000001, 'MISSOURI': 923640.9999999998, 'MONTANA': 257005.99999999994, 'NEBRASKA': 254159.0, 'NEVADA': 685073.0, 'NEW HAMPSHIRE': 193436.0, 'NEW JERSEY': 1759653.0, 'NEW YORK': 6263164.000000002, 'NORTH CAROLINA': 1851557.999999999, 'NORTH DAKOTA': 96756.99999999999, 'OHIO': 2788133.9999999995, 'OKLAHOMA': 797219.9999999998, 'OREGON

# 1 million CHW allocation to states

We allocate 1 million CHWs to states proportional to total Medicaid enrolles in each state.

Let's $FedCHW$ represents the number of CHW will be allocated within states by the federal government, which is 1 million in our project. $TotMed$ represents the total Medicaid enrollee numbers over the US, $Med_s$ is the total Medicaid enrollee numbers in state $s \in S$, and $CHW_s$ is the total number of CHW allocated to state $ s\in S$. 

$CHW_s = FedCHW*\frac{Med_s}{TotMed}$



In [94]:
# We consider allocation of 1 million CHW all over the US
Federal_budget_CHW = 1000000

# First, we calculate the Total Medicaid enrolles all over the US
Total_federal_need = sum(Medicaid_demand_state[s] for s in State)

# Allocate the 1 million CHWs proportional to Medicaid enrolles in each state
Medicaid_budget_state = {s: (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW  for s in State}

In [95]:
for s in State:
    print (s, Medicaid_demand_state[s], Total_federal_need, Federal_budget_CHW, (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW)

NEW MEXICO 772102.0 76265716.0 1000000 10123.841228999934
ALABAMA 957116.0000000001 76265716.0 1000000 12549.754335224496
ALASKA 231145.0 76265716.0 1000000 3030.7851564653242
ARIZONA 1839932.0 76265716.0 1000000 24125.283239981647
ARKANSAS 830467.0000000003 76265716.0 1000000 10889.126117953188
CALIFORNIA 11847711.0 76265716.0 1000000 155347.79742971272
COLORADO 1337805.0 76265716.0 1000000 17541.368129291543
CONNECTICUT 874974.0 76265716.0 1000000 11472.70419647014
DELAWARE 239009.0 76265716.0 1000000 3133.8983298865246
FLORIDA 3892551.999999999 76265716.0 1000000 51039.342500895145
GEORGIA 1928702.999999999 76265716.0 1000000 25289.25316848791
IDAHO 340742.0000000001 76265716.0 1000000 4467.8266706366485
ILLINOIS 2987496.0000000005 76265716.0 1000000 39172.20156957552
INDIANA 1602975.9999999995 76265716.0 1000000 21018.30395193562
IOWA 699741.0 76265716.0 1000000 9175.040066495934
KANSAS 401103.0000000001 76265716.0 1000000 5259.283214491818
KENTUCKY 1416013.0000000002 76265716.0 10

In [96]:
print (Covid_state)

{'NEW MEXICO': 15225.0, 'ALABAMA': 45877.0, 'ALASKA': 3298.0, 'ARIZONA': 79460.0, 'ARKANSAS': 29327.0, 'CALIFORNIA': 498933.0, 'COLORADO': 30038.0, 'CONNECTICUT': 22923.0, 'DELAWARE': 7912.0, 'FLORIDA': 130089.0, 'GEORGIA': 92095.0, 'IDAHO': 12057.0, 'ILLINOIS': 70375.0, 'INDIANA': 60007.0, 'IOWA': 15725.0, 'KANSAS': 26725.0, 'KENTUCKY': 26529.0, 'LOUISIANA': 32888.0, 'MAINE': 5485.0, 'MARYLAND': 31462.0, 'MASSACHUSETTS': 52310.0, 'MICHIGAN': 30417.0, 'MINNESOTA': 20646.0, 'MISSISSIPPI': 26275.0, 'MISSOURI': 33784.0, 'MONTANA': 5196.0, 'NEBRASKA': 12065.0, 'NEVADA': 24887.0, 'NEW HAMPSHIRE': 8194.0, 'NEW JERSEY': 55506.0, 'NEW YORK': 136756.0, 'NORTH CAROLINA': 70290.0, 'NORTH DAKOTA': 3023.0, 'OHIO': 85952.0, 'OKLAHOMA': 34903.0, 'OREGON': 13568.0, 'PENNSYLVANIA': 99852.0, 'RHODE ISLAND': 10526.0, 'SOUTH CAROLINA': 36970.0, 'SOUTH DAKOTA': 4835.0, 'TENNESSEE': 72190.0, 'TEXAS': 195388.0, 'UTAH': 27794.0, 'VERMONT': 1071.0, 'VIRGINIA': 51831.0, 'WASHINGTON': 25041.0, 'WEST VIRGINIA': 1

# Proportional allocation for different vulnerability values

Let V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}. We assume $v_j$ represent the vulnerability value for county $j \in J$, while $v_s$ represent the sum of the vulnerability values for each county in the state of county j.

$Prop_{v_j} = \frac{v_j}{v_s}*CHW_s$



In [97]:
# Calling proportional allocation function for different vulnerability criterias

# Proportional allocation according to cumulative Covid death in per capita in each county
Proportional_to_covid_death_cap = Proportional_allocation(Covid_death_capita, Total_covid_death_cap,Medicaid_budget_state)

# Propportional allocation according to Medicaid enrollee number in each county
Proportional_to_medicaid = Proportional_allocation(Medicaid_demand, Medicaid_demand_state,Medicaid_budget_state )

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_medicaid_cap = Proportional_allocation(Medicaid_capita, Total_medicaid_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases in each county
Proportional_to_covid = Proportional_allocation(COVID_14days, Covid_state, Medicaid_budget_state)

# Proportional allocation according to SVI score in each county
Proportional_to_SVI = Proportional_allocation(SVI_county, SVI_state, Medicaid_budget_state)

# Proportional allocation according to YPLL in each county
Proportional_to_YPLL = Proportional_allocation(YPLL, YPLL_state, Medicaid_budget_state)

# Proportional allocation according to Unemployment  in each county
Proportional_to_unemployment = Proportional_allocation(Unemployment, Unemployment_state, Medicaid_budget_state)

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_unemployment_cap = Proportional_allocation(Unemployment_capita, Total_unemployment_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases per capita in each county
Proportional_to_covid_capita = Proportional_allocation(Covid_capita, Total_covid_cap, Medicaid_budget_state)



# Normalize values for comparison
To be able compare the different vulnerability values for each county, we normalize all vulnerability values as follows. 

Let $m_{v_s} = \min \{v_j, \text{ for county j in state s }\}$  and 
$M_{v_s} = \max \{v_j, \text{ for county j in state s }\}$.

We calculate the normalize value for each vulnerability for each county by substracting the min vulnerability in the state of the county and dividing that by the differences between max and min value of the vulnerability values in the state. Mathematical formulation for the normalization is as follows.

$N_{v_j} = \frac{v_j - m_{v_s}}{M_{v_s} - m_{v_s}}$ 

for each $v \in V$, where V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}, j is a county in each state $s \in S$.


In [98]:
#Normalize function to normalize the vulnerability values to be able to compare them

def normalize(dict_1):
    
    result = {}
    min_data = {s: min(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    max_data = {s: max(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    
    for (j,s) in cartesian_pro_county_state:
        
        if (max_data[s] - min_data[s]) != 0 :
    
            result[j] = (dict_1[j] - min_data[s])/(max_data[s] - min_data[s])
        
        else:
            result[j] = 1
    
    return result 

# Percentile Rank

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html"> The function scipy.stats.percentileofscore (a, score, kind='rank')   </a>
computes the percentile rank of a score relative to a list of scores. 
"rank": Average percentage ranking of score. In case of multiple matches, average the percentage rankings of all matching scores.

In [99]:
from scipy import stats

# Calculate percentile ranks

def percentile_ranks(data):
    x = {s: [] for s in State}

    for (j,s) in cartesian_pro_county_state:
         
        x[s].append(data[j])
    
    
    
    percentile_ranks = {i: stats.percentileofscore(x[s], data[i], 'rank') for (i,s) in cartesian_pro_county_state}

    return percentile_ranks

In [100]:
# Write timestamp 

time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

In [101]:
#print (Medicaid_demand)

In [102]:
#Write a function to order the dicts
def order_k(dict_1):
    dict_2 = {}
    for m in location:
        if m in dict_1.keys():
            dict_2[m] = dict_1[m]
        else:
            dict_2[m] = 0
    
    return dict_2
            

In [103]:


Medicaid_demand = order_k(Medicaid_demand)#{m: Medicaid_demand[m] for m in location}
COVID_14days = order_k(COVID_14days)#{m: COVID_14days[m] for m in location}
SVI_county = order_k(SVI_county)#{m: SVI_county[m] for m in location}
YPLL = order_k(YPLL)#{m: YPLL[m] for m in location}
Unemployment = order_k(Unemployment)#{m: Unemployment[m] for m in location}
Unemployment_capita = order_k(Unemployment_capita)#{m: Unemployment_capita[m] for m in location}

In [104]:
# Write file allocation with each strategies for each county 

Strategies = ["Medicaid_demand", "Medicaid_capita", "Covid", "SVI"
              , "YPLL","Unemployment", "Unemployment_capita", "Covid_capita",  "Covid_death_capita" ]

fieldnames = []  
fieldnames.append('County_FIPS')


SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    fieldnames.append('Proportional_allocation_to_' + s)
    fieldnames.append(s)
    fieldnames.append('Percentile_ranks_' + s)

    
        

writefile = 'Output/County_level_proportional_allocation_for_all_policies.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location
                   , Proportional_to_medicaid.values(),         Medicaid_demand.values(),      percentile_ranks(Medicaid_demand).values()
                   , Proportional_to_medicaid_cap.values(),     Medicaid_capita.values(),      percentile_ranks(Medicaid_capita).values()
                   , Proportional_to_covid.values(),            COVID_14days.values(),         percentile_ranks(COVID_14days).values()
                   , Proportional_to_SVI.values(),              SVI_county.values(),           percentile_ranks(SVI_county).values()
                   , Proportional_to_YPLL.values(),             YPLL.values(),                 percentile_ranks(YPLL).values()
                   , Proportional_to_unemployment.values(),     Unemployment.values(),         percentile_ranks(Unemployment).values()
                   , Proportional_to_unemployment_cap.values(), Unemployment_capita.values(),  percentile_ranks(Unemployment_capita).values()
                   , Proportional_to_covid_capita.values(),     Covid_capita.values(),         percentile_ranks(Covid_capita).values()
                   , Proportional_to_covid_death_cap.values(),  Covid_death_capita.values(),   percentile_ranks(Covid_death_capita).values() ):                    
       
        writer.writerow(row)

In [105]:
writefile = 'Output/State_level_allocation.csv'

cl = ['State', 'CHW_allocation']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(cl)
    for row in zip( State, Medicaid_demand_state.values()):
        writer.writerow(row)